In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import json
file = open('heavenli.json', 'r')
# file = open('xnli.json', 'r')
data = [json.loads(line.strip()) for line in file]
file.close()

# SOTA pre-trained model

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = model.to(device)


In [11]:
dev_predictions = []
for e in data:
    premise = e['en_sentence']['text']
    hypothesis = e['vi_sentence']['text']

    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = ["entailment", "neutral", "contradiction"]
    prediction = [(name,round(float(pred) * 100, 1)) for pred, name in zip(prediction, label_names)]
    prediction.sort(key=lambda x: x[1])
    dev_predictions.append(prediction[-1][0])
    print(len(dev_predictions))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


KeyboardInterrupt: 

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score([e['label'].lower() for e in data[:len(dev_predictions)]], dev_predictions)


0.71875